In [4]:
import psycopg2  
import pprint
import pandas as pd


In [5]:
# on your computer you are the user postgres (full administrative access)
db_user = 'postgres'
# if you need a password to access a database, put it here
db_password = ''
# on your computer, use localhost
db_host = 'localhost'
# the default port for postgres is 5432
db_port = 5432
# we want to connect to the northwind database
database = 'postgres'

conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
conn = psycopg2.connect(conn_str)

In [6]:
def sql(query, con=conn):
    df = pd.read_sql(query, con=conn)
    return df

In [7]:
query = """ SELECT * 
FROM match
WHERE date >'2000/01/01'
"""


all_matches = sql(query)

In [8]:
query = """ SELECT * 
FROM player
"""


all_playas = sql(query)

In [9]:
query = """ SELECT * 
FROM player_season_surface_stats
"""


players_stats = sql(query)

In [10]:
query = """ SELECT * 
FROM player_season_stats
"""


players_stats_season = sql(query)

In [11]:
players_stats_season.to_csv('player_season_stats.csv',index=False)

In [12]:
query = """ SELECT *
FROM tournament_event
"""
tournament_event2 =  sql(query)


In [13]:
# Baseline Model - selecting highest ranked player
baseline = len(all_matches[all_matches['winner_rank']<all_matches['loser_rank']])/len(all_matches)
print("Baseline Selection is {:.3f}%".format(baseline))

Baseline Selection is 0.647%


In [14]:
all_matches.columns

Index(['match_id', 'tournament_event_id', 'match_num', 'date', 'surface',
       'indoor', 'round', 'best_of', 'winner_id', 'winner_country_id',
       'winner_seed', 'winner_entry', 'winner_rank', 'winner_rank_points',
       'winner_elo_rating', 'winner_next_elo_rating', 'winner_age',
       'winner_height', 'loser_id', 'loser_country_id', 'loser_seed',
       'loser_entry', 'loser_rank', 'loser_rank_points', 'loser_elo_rating',
       'loser_next_elo_rating', 'loser_age', 'loser_height', 'score',
       'outcome', 'w_sets', 'l_sets', 'w_games', 'l_games', 'w_tbs', 'l_tbs',
       'has_stats'],
      dtype='object')

In [15]:
all_playas.columns

Index(['player_id', 'first_name', 'last_name', 'dob', 'dod', 'country_id',
       'birthplace', 'residence', 'height', 'weight', 'hand', 'backhand',
       'active', 'turned_pro', 'coach', 'prize_money', 'wikipedia', 'web_site',
       'facebook', 'twitter', 'nicknames'],
      dtype='object')

In [16]:
players_stats.columns

Index(['player_id', 'season', 'surface', 'p_matches', 'o_matches', 'p_sets',
       'o_sets', 'p_games', 'o_games', 'p_tbs', 'o_tbs', 'p_ace', 'p_df',
       'p_sv_pt', 'p_1st_in', 'p_1st_won', 'p_2nd_won', 'p_sv_gms', 'p_bp_sv',
       'p_bp_fc', 'o_ace', 'o_df', 'o_sv_pt', 'o_1st_in', 'o_1st_won',
       'o_2nd_won', 'o_sv_gms', 'o_bp_sv', 'o_bp_fc', 'minutes',
       'matches_w_stats', 'sets_w_stats', 'games_w_stats', 'opponent_rank',
       'opponent_elo_rating', 'p_upsets', 'o_upsets', 'matches_w_rank'],
      dtype='object')

In [17]:
tournament_event2.columns

Index(['tournament_event_id', 'tournament_id', 'original_tournament_id',
       'season', 'date', 'name', 'city', 'level', 'surface', 'indoor',
       'draw_type', 'draw_size', 'rank_points', 'map_properties'],
      dtype='object')

In [18]:
# for all data create
# matches filtered
matches_filtered = all_matches.copy()
# matches_filtered.drop(['w_sets', 'l_sets', 'w_games', 'l_games', 'w_tbs', 'l_tbs'],axis=1,inplace=True)
matches_filtered['p1_combination'] = matches_filtered['winner_id'].astype(str)+"_"+matches_filtered['loser_id'].astype(str)
matches_filtered['p2_combination'] = matches_filtered['loser_id'].astype(str)+"_"+matches_filtered['winner_id'].astype(str)
# dropping all matches not finished
df_drop = matches_filtered.dropna(subset=['outcome'])
matches_filtered = matches_filtered.drop(df_drop.index)
# player1 details
player1_deets = all_playas.copy()
player1_deets.drop(['country_id','birthplace', 'residence', 'height','wikipedia', 'web_site','facebook', 'twitter'],axis=1,inplace=True)
player1_deets=player1_deets.add_prefix('p1_')
# player2 details
player2_deets = all_playas.copy()
player2_deets.drop(['country_id','birthplace', 'residence', 'height','wikipedia', 'web_site','facebook', 'twitter'],axis=1,inplace=True)
player2_deets=player2_deets.add_prefix('p2_')
# player1 stats
player1_stats = players_stats.copy()
player1_stats.drop(['sets_w_stats', 'games_w_stats'],axis=1,inplace=True)
player1_stats=player1_stats.add_prefix('p1_')
# player2 stats
player2_stats = players_stats.copy()
player2_stats.drop(['sets_w_stats', 'games_w_stats'],axis=1,inplace=True)
player2_stats=player2_stats.add_prefix('p2_')
# player1 tournament wins and player2
query = """ SELECT * 
FROM player_season_performance
"""
player1_perf = sql(query)
player2_perf = player1_perf.copy()
player1_perf=player1_perf.add_prefix('p1_')
player2_perf=player2_perf.add_prefix('p2_')
# tournament classification
query = """ SELECT tournament_event_id, t.tournament_id, season, t.name, t.city,t.level,country_id
FROM tournament_event t
INNER JOIN tournament t2
ON t.tournament_id = t2.tournament_id
"""
tournament_event =  sql(query)

# match stats
query = """ SELECT * 
FROM match_stats
"""
match_stats = sql(query)








In [19]:
player1_stats.columns



Index(['p1_player_id', 'p1_season', 'p1_surface', 'p1_p_matches',
       'p1_o_matches', 'p1_p_sets', 'p1_o_sets', 'p1_p_games', 'p1_o_games',
       'p1_p_tbs', 'p1_o_tbs', 'p1_p_ace', 'p1_p_df', 'p1_p_sv_pt',
       'p1_p_1st_in', 'p1_p_1st_won', 'p1_p_2nd_won', 'p1_p_sv_gms',
       'p1_p_bp_sv', 'p1_p_bp_fc', 'p1_o_ace', 'p1_o_df', 'p1_o_sv_pt',
       'p1_o_1st_in', 'p1_o_1st_won', 'p1_o_2nd_won', 'p1_o_sv_gms',
       'p1_o_bp_sv', 'p1_o_bp_fc', 'p1_minutes', 'p1_matches_w_stats',
       'p1_opponent_rank', 'p1_opponent_elo_rating', 'p1_p_upsets',
       'p1_o_upsets', 'p1_matches_w_rank'],
      dtype='object')

In [20]:
player1_perf

,p1_player_id,p1_season,p1_matches_won,p1_matches_lost,p1_grand_slam_matches_won,p1_grand_slam_matches_lost,p1_tour_finals_matches_won,p1_tour_finals_matches_lost,p1_alt_finals_matches_won,p1_alt_finals_matches_lost,...,p1_vs_top10_won,p1_vs_top10_lost,p1_after_winning_first_set_won,p1_after_winning_first_set_lost,p1_after_losing_first_set_won,p1_after_losing_first_set_lost,p1_tie_breaks_won,p1_tie_breaks_lost,p1_deciding_set_tbs_won,p1_deciding_set_tbs_lost
0,1,1968,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,1.0,NaN,NaN,0.0,0.0,NaN,NaN
1,1,1977,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,NaN,NaN
2,2,1968,2,2,1.0,1.0,NaN,NaN,NaN,NaN,...,0.0,2.0,2.0,1.0,0.0,1.0,0.0,0.0,NaN,NaN
3,2,1969,2,5,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,2.0,2.0,0.0,0.0,5.0,0.0,0.0,NaN,NaN
4,2,1970,1,4,1.0,1.0,NaN,NaN,NaN,NaN,...,0.0,3.0,1.0,1.0,0.0,3.0,1.0,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26338,53992,1969,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,1.0,NaN,NaN,0.0,1.0,0.0,0.0,NaN,NaN
26339,53994,1970,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,NaN,NaN
26340,53995,1971,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,NaN,NaN
26341,53995,1978,0,1,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,NaN,NaN


In [21]:
# join all the data

# add player deets
all_data = matches_filtered.join(player1_deets.set_index('p1_player_id'), on='winner_id')
all_data = all_data.join(player2_deets.set_index('p2_player_id'), on='loser_id')
# add in tournament information
all_data = all_data.join(tournament_event.set_index('tournament_event_id'),on='tournament_event_id')
all_data["last_season"] = all_data.season -1

# add in previous season stats for each surface
all_data = pd.merge(all_data, player1_stats, how='left', left_on=['winner_id','surface','last_season'], right_on = ['p1_player_id','p1_surface','p1_season'])
all_data = pd.merge(all_data, player2_stats, how='left', left_on=['loser_id','surface','last_season'], right_on = ['p2_player_id','p2_surface','p2_season'])

# add in the past tournament specific wins per season
all_data = pd.merge(all_data, player1_perf, how='left', left_on=['winner_id','last_season'], right_on = ['p1_player_id','p1_season'])
all_data = pd.merge(all_data, player2_perf, how='left', left_on=['loser_id','last_season'], right_on = ['p2_player_id','p2_season'])

# add in all previous season stats



# add in all previous tournament wins



In [22]:
for col in all_data:
    print((col,all_data[col][10]))

('match_id', 110786)
('tournament_event_id', 2704)
('match_num', 26)
('date', datetime.date(2000, 2, 21))
('surface', 'C')
('indoor', False)
('round', 'QF')
('best_of', 3)
('winner_id', 2905)
('winner_country_id', 'AUT')
('winner_seed', 4.0)
('winner_entry', None)
('winner_rank', 38.0)
('winner_rank_points', 882.0)
('winner_elo_rating', 1938.0)
('winner_next_elo_rating', 1946.0)
('winner_age', 23.13)
('winner_height', 175.0)
('loser_id', 3292)
('loser_country_id', 'ARG')
('loser_seed', nan)
('loser_entry', None)
('loser_rank', 65.0)
('loser_rank_points', 627.0)
('loser_elo_rating', 1849.0)
('loser_next_elo_rating', 1839.0)
('loser_age', 21.2)
('loser_height', 175.0)
('score', '7-6(5) 6-4')
('outcome', None)
('w_sets', 2.0)
('l_sets', 0.0)
('w_games', 12.0)
('l_games', 10.0)
('w_tbs', 1.0)
('l_tbs', 0.0)
('has_stats', True)
('p1_combination', '2905_3292')
('p2_combination', '3292_2905')
('p1_first_name', 'Stefan')
('p1_last_name', 'Koubek')
('p1_dob', datetime.date(1977, 1, 2))
('p1_dod

In [23]:
# player1_stats_all = pd.DataFrame(player1_stats[(player1_stats['p1_player_id']==2)&(player1_stats['p1_season']<1969)&(player1_stats['p1_surface']=='H')].sum()).T


In [24]:
# # creating a new dataframe with a players stats before that date

# player1_stats_all = pd.DataFrame(columns=player1_stats.columns)

# for ind in player1_stats.index:
#         player1_stats_all = player1_stats_all.append(pd.DataFrame(player1_stats[(player1_stats['p1_player_id']==player1_stats['p1_player_id'].iloc[ind]) \
#                                                                  &(player1_stats['p1_season']<player1_stats['p1_season'].iloc[ind]) \
#                                                                  &(player1_stats['p1_surface']==player1_stats['p1_surface'].iloc[ind])].sum()).T)
#         player1_stats_all['p1_player_id'].iloc[ind] = player1_stats['p1_player_id'].iloc[ind]
#         player1_stats_all['p1_season'].iloc[ind] = player1_stats['p1_season'].iloc[ind]
#         player1_stats_all['p1_surface'].iloc[ind] = player1_stats['p1_surface'].iloc[ind]
#         if ind % 1000 == 0:
#             print("done -",ind)
# player1_stats_all = player1_stats_all.add_suffix('_all')            


In [25]:
player1_stats_all = pd.read_csv("p_stats_for_all_previous_seasons.csv")

In [26]:
player1_stats_all

,p1_player_id_all,p1_season_all,p1_surface_all,p1_p_matches_all,p1_o_matches_all,p1_p_sets_all,p1_o_sets_all,p1_p_games_all,p1_o_games_all,p1_p_tbs_all,...,p1_o_sv_gms_all,p1_o_bp_sv_all,p1_o_bp_fc_all,p1_minutes_all,p1_matches_w_stats_all,p1_opponent_rank_all,p1_opponent_elo_rating_all,p1_p_upsets_all,p1_o_upsets_all,p1_matches_w_rank_all
0,1.0,1968.0,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,1.0,1977.0,C,0.0,1.0,1.0,2.0,14.0,16.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.313220,1500.0,0.0,0.0,0.0
2,2.0,1968.0,H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,2.0,1968.0,G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,2.0,1969.0,H,1.0,1.0,3.0,2.0,23.0,18.0,0.0,...,0.0,0.0,0.0,0.0,0.0,9.392662,3618.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52498,53992.0,1969.0,G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
52499,53994.0,1970.0,G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
52500,53995.0,1971.0,C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
52501,53995.0,1978.0,H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


In [27]:
# player1_stats_all.to_csv("p_stats_for_all_previous_seasons.csv" , index = False)

In [28]:
# create p2 from p1
player2_stats_all = player1_stats_all.copy()

player2_stats_all.columns = ['p2_player_id', 'p2_season', 'p2_surface', 'p2_p_matches',
       'p2_o_matches', 'p2_p_sets', 'p2_o_sets', 'p2_p_games', 'p2_o_games',
       'p2_p_tbs', 'p2_o_tbs', 'p2_p_ace', 'p2_p_df', 'p2_p_sv_pt',
       'p2_p_1st_in', 'p2_p_1st_won', 'p2_p_2nd_won', 'p2_p_sv_gms',
       'p2_p_bp_sv', 'p2_p_bp_fc', 'p2_o_ace', 'p2_o_df', 'p2_o_sv_pt',
       'p2_o_1st_in', 'p2_o_1st_won', 'p2_o_2nd_won', 'p2_o_sv_gms',
       'p2_o_bp_sv', 'p2_o_bp_fc', 'p2_minutes', 'p2_matches_w_stats',
       'p2_opponent_rank', 'p2_opponent_elo_rating', 'p2_p_upsets',
       'p2_o_upsets', 'p2_matches_w_rank']

player2_stats_all = player2_stats_all.add_suffix('_all')
# player1_stats_all = player1_stats_all.add_suffix('_all')

In [29]:
all_data[['winner_id','season','surface']].dtypes

winner_id     int64
season        int64
surface      object
dtype: object

In [30]:
player1_stats_all[['p1_surface_all']]

,p1_surface_all
0,C
1,C
2,H
3,G
4,H
...,...
52498,G
52499,G
52500,C
52501,H


In [31]:
len(all_data)

59454

In [32]:
# join this into the all data!

all_data = pd.merge(all_data, player1_stats_all, how='left', left_on=['winner_id','season','surface'], right_on = ['p1_player_id_all','p1_season_all','p1_surface_all'])
all_data = pd.merge(all_data, player2_stats_all, how='left', left_on=['loser_id','season','surface'], right_on = ['p2_player_id_all','p2_season_all','p2_surface_all'])



In [33]:
len(all_data)

59454

In [34]:
win_ratio_season_df = pd.read_csv('win_ratio_template.csv')

In [35]:
# loop through and find rivalries for all seasons

for season in all_data.season.unique():
    win_ratio = pd.DataFrame(all_data['p1_combination'][all_data['season']==season].value_counts())
    lose_ratio = pd.DataFrame(all_data['p2_combination'][all_data['season']==season].value_counts())
    win_ratio = win_ratio.join(lose_ratio,how='outer')
    win_ratio.fillna(0, inplace=True)
    win_ratio['p1_pct_overall'] = win_ratio['p1_combination']/(win_ratio['p1_combination']+win_ratio['p2_combination'])
    win_ratio['p2_pct_overall'] = win_ratio['p2_combination']/(win_ratio['p1_combination']+win_ratio['p2_combination'])
    win_ratio['match_win_diff'] = win_ratio['p1_combination']-win_ratio['p2_combination']
    win_ratio['rivalry_total'] = win_ratio['p1_combination']+win_ratio['p2_combination']
    win_ratio.rename(columns={'p1_combination':'p1_beat_p2_overall','p2_combination':'p2_beat_p1_overall'},inplace=True)
    win_ratio['season_w_ratio'] = season
    win_ratio_season_df = pd.concat([win_ratio_season_df,win_ratio],sort=False)
all_data = pd.merge(all_data, win_ratio_season_df, how='left', left_on=['p1_combination','last_season'], right_on=[win_ratio_season_df.index,'season_w_ratio'])



In [36]:
# loop through and find rivalries for all previous seasons

win_ratio_season_df = pd.read_csv('win_ratio_template.csv')
for season in all_data.season.unique():
    win_ratio = pd.DataFrame(all_data['p1_combination'][all_data['season']<=season].value_counts())
    lose_ratio = pd.DataFrame(all_data['p2_combination'][all_data['season']<=season].value_counts())
    win_ratio = win_ratio.join(lose_ratio,how='outer')
    win_ratio.fillna(0,inplace=True)
    win_ratio['p1_pct_overall'] = win_ratio['p1_combination']/(win_ratio['p1_combination']+win_ratio['p2_combination'])
    win_ratio['p2_pct_overall'] = win_ratio['p2_combination']/(win_ratio['p1_combination']+win_ratio['p2_combination'])
    win_ratio['match_win_diff'] = win_ratio['p1_combination']-win_ratio['p2_combination']
    win_ratio['rivalry_total'] = win_ratio['p1_combination']+win_ratio['p2_combination']
    win_ratio.rename(columns={'p1_combination':'p1_beat_p2_overall','p2_combination':'p2_beat_p1_overall'},inplace=True)
    win_ratio['season_w_ratio'] = season
    win_ratio_season_df = pd.concat([win_ratio_season_df,win_ratio],sort=False)
win_ratio_season_df_all=win_ratio_season_df.add_suffix('_all')
all_data = pd.merge(all_data, win_ratio_season_df_all, how='left', left_on=['p1_combination','last_season'], right_on=[win_ratio_season_df.index,'season_w_ratio_all'])


In [38]:
all_data

,match_id,tournament_event_id,match_num,date,surface,indoor,round,best_of,winner_id,winner_country_id,...,match_win_diff,rivalry_total,season_w_ratio,p1_beat_p2_overall_all,p2_beat_p1_overall_all,p1_pct_overall_all,p2_pct_overall_all,match_win_diff_all,rivalry_total_all,season_w_ratio_all
0,110583,2698,25,2000-02-07,H,False,QF,3,3017,GER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110584,2698,26,2000-02-07,H,False,QF,3,2610,ESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110585,2698,27,2000-02-07,H,False,QF,3,3507,ESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,110586,2698,28,2000-02-07,H,False,QF,3,2223,MAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,110587,2698,29,2000-02-07,H,False,SF,3,3017,GER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59449,174488,4176,82,2020-01-03,H,False,SF,3,4742,ESP,...,1.0,1.0,2019,2.0,0.0,1.000000,0.000000,2.0,2.0,2019
59450,174489,4176,83,2020-01-03,H,False,SF,3,5571,SRB,...,NaN,NaN,NaN,2.0,0.0,1.000000,0.000000,2.0,2.0,2019
59451,174490,4176,84,2020-01-03,H,False,SF,3,4920,SRB,...,-1.0,3.0,2019,2.0,2.0,0.500000,0.500000,0.0,4.0,2019
59452,174491,4176,85,2020-01-03,H,False,F,3,5131,ESP,...,NaN,NaN,NaN,3.0,0.0,1.000000,0.000000,3.0,3.0,2019


In [39]:
all_data.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59454 entries, 0 to 59453
Data columns (total 344 columns):
match_id                           59454 non-null int64
tournament_event_id                59454 non-null int64
match_num                          59454 non-null int64
date                               59454 non-null object
surface                            59268 non-null object
indoor                             59454 non-null bool
round                              59454 non-null object
best_of                            59454 non-null int64
winner_id                          59454 non-null int64
winner_country_id                  59454 non-null object
winner_seed                        24783 non-null float64
winner_entry                       7058 non-null object
winner_rank                        59152 non-null float64
winner_rank_points                 59152 non-null float64
winner_elo_rating                  59454 non-null float64
winner_next_elo_rating             5945

In [40]:
# all_data[['date','p1_last_name','p2_last_name']]
all_data['lookup'] = all_data['p1_last_name'].apply(lambda x: x.partition(' ')[0])+all_data['p2_last_name'].apply(lambda x: x.partition(' ')[0])+all_data.date.apply(lambda x: str(x))

# all_data['lookup'] = all_data['p1_last_name']+all_data['p2_last_name']+all_data.date.apply(lambda x: str(x))

all_data['lookup'] = all_data['lookup'].apply(lambda x: x[:-3])


In [41]:
# import all the odds information

# path = r'/Users/lukebetham/Documents/DSI_General_Assembly/Capstone Project/Tennis/mens/'                     # use your path
# all_files = glob.glob(os.path.join(path, "*.xls*"))     # advisable to use os.path.join as this makes concatenation OS independent

# df_from_each_file = (pd.read_excel(f) for f in all_files)
# concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)

In [42]:
# concatenated_df= pd.read_csv('/Users/lukebetham/Documents/DSI_General_Assembly/Capstone Project/Tennis/mens/agg.csv')
# concatenated_df.drop('Unnamed: 0',axis=1,inplace=True)

In [43]:
# concatenated_df.columns



In [44]:
# bet_df = concatenated_df.drop(['ATP','Best of','Comment', 'Court', 'L1', 'L2', 'L3', 'L4', 'L5','LPts',
#                                'LRank', 'Location', 'Lsets','Round','Series', 'Surface', 'W1', 'W2', 'W3',
#                                'W4', 'W5', 'WPts','WRank','Wsets','B&WL', 'B&WW','AvgL', 'AvgW','CBL', 'CBW',
#                               'GBL', 'GBW','IWL', 'IWW','MaxL', 'MaxW','SBL', 'SBW', 'SJL', 'SJW','UBL', 'UBW'],axis=1)

In [45]:
# bet_df = bet_df.sort_values('Date').reset_index(drop=True)

In [46]:
# # Getting the average and odds ranges
# bet_df['w_mean'] = bet_df[['B365W','EXW','LBW','PSW']].mean(axis=1,skipna=True)
# bet_df['l_mean'] = bet_df[['B365L','EXL','LBL','PSL']].mean(axis=1,skipna=True)
# bet_df['w_max'] = bet_df[['B365W','EXW','LBW','PSW']].max(axis=1,skipna=True)
# bet_df['l_max'] = bet_df[['B365L','EXL','LBL','PSL']].max(axis=1,skipna=True)
# bet_df['w_min'] = bet_df[['B365W','EXW','LBW','PSW']].min(axis=1,skipna=True)
# bet_df['l_min'] = bet_df[['B365L','EXL','LBL','PSL']].min(axis=1,skipna=True)

# # Dropping rows with no odds
# bet_df.dropna(subset=['w_mean'],inplace=True)
# bet_df.dropna(subset=['l_mean'],inplace=True)

# # converting the odds into probabilites
# bet_df['w_mean_prob'] = 1/bet_df['w_mean']
# bet_df['l_mean_prob'] = 1/bet_df['l_mean']
# bet_df['mean_margin'] = 1-bet_df['w_mean_prob']-bet_df['l_mean_prob'] 


In [47]:
# bet_df.columns

In [48]:
# bet_df_final = bet_df[['Date','Winner','Loser','Tournament','w_mean', 'l_mean', 'w_max', 'l_max','w_min', 'l_min',
#                        'w_mean_prob', 'l_mean_prob', 'mean_margin']]


In [49]:
# bet_df_final.to_csv('/Users/lukebetham/Documents/DSI_General_Assembly/Capstone Project/Tennis/mens/usable_odds.csv')

In [50]:
bet_df_final = pd.read_csv('/Users/lukebetham/Documents/DSI_General_Assembly/Capstone Project/Tennis/mens/usable_odds.csv')

In [51]:
bet_df_final.drop('Unnamed: 0',axis=1,inplace=True)
bet_df_final['lookup'] = bet_df_final['Winner'].apply(lambda x: x.partition(' ')[0])+bet_df_final['Loser'].apply(lambda x: x.partition(' ')[0])+bet_df_final.Date
bet_df_final['lookup'] = bet_df_final['lookup'].apply(lambda x: x[:-3])

In [52]:
# bet_df_final[bet_df_final.duplicated(subset=['lookup'])].sort_values("Date")

In [53]:
all_data = all_data[~all_data.duplicated(subset=['lookup'])]

In [54]:
len(all_data)

58676

In [55]:
bet_df_final = bet_df_final[~bet_df_final.duplicated(subset=['lookup'])]

In [56]:
# add in the odds file
all_data = all_data.join(bet_df_final.set_index("lookup"),on='lookup')


In [57]:
len(all_data)

58676

In [58]:
all_data.info(verbose=True,null_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 58676 entries, 0 to 59453
Data columns (total 358 columns):
match_id                           58676 non-null int64
tournament_event_id                58676 non-null int64
match_num                          58676 non-null int64
date                               58676 non-null object
surface                            58491 non-null object
indoor                             58676 non-null bool
round                              58676 non-null object
best_of                            58676 non-null int64
winner_id                          58676 non-null int64
winner_country_id                  58676 non-null object
winner_seed                        24318 non-null float64
winner_entry                       7025 non-null object
winner_rank                        58377 non-null float64
winner_rank_points                 58377 non-null float64
winner_elo_rating                  58676 non-null float64
winner_next_elo_rating             5867

In [59]:
all_data.to_csv('/Users/lukebetham/Documents/DSI_General_Assembly/Capstone Project/Tennis/all_matches_all_data_2000.csv',index=False)

In [60]:
all_data

,match_id,tournament_event_id,match_num,date,surface,indoor,round,best_of,winner_id,winner_country_id,...,Tournament,w_mean,l_mean,w_max,l_max,w_min,l_min,w_mean_prob,l_mean_prob,mean_margin
0,110583,2698,25,2000-02-07,H,False,QF,3,3017,GER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,110584,2698,26,2000-02-07,H,False,QF,3,2610,ESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,110585,2698,27,2000-02-07,H,False,QF,3,3507,ESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,110586,2698,28,2000-02-07,H,False,QF,3,2223,MAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,110587,2698,29,2000-02-07,H,False,SF,3,3017,GER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59449,174488,4176,82,2020-01-03,H,False,SF,3,4742,ESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59450,174489,4176,83,2020-01-03,H,False,SF,3,5571,SRB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59451,174490,4176,84,2020-01-03,H,False,SF,3,4920,SRB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59452,174491,4176,85,2020-01-03,H,False,F,3,5131,ESP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
